# Snowflake Cortex AI - Transforming & Semantic Model Generation

This notebook shows how you can use Large Language Models to help you transforming from raw, bronze to gold layer. You can then generate a semantic model in a separate quickstart where users can ask questions using natural language. The bronze layer will have tables with column names that meaningful to Analyst. The Gold Layer will have the views that will be used by Cortex Analyst, Snowflake text-2-sql capability. 

In [ ]:
# Import python packages
import streamlit as st

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
session

Change this definition in case you want to use other names or LLMs. Note this has been tested using the Anthropic model claude-3-5-sonnet. For LLM Region availability check: https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions or enable Cross-Region Support.

In [ ]:
LLM = 'claude-3-5-sonnet'
RAW_LAYER = 'SAP_RAW'
BRONZE_LAYER = 'SAP_BRONZE'
GOLD_LAYER = 'SAP_GOLD'

Note this Notebook is expected to run in a test database and it will create a clean environment. This is replacing the BRONZE_LAYER schema in case it exists!!

In [ ]:
session.sql(f'create or replace schema {BRONZE_LAYER}').collect()

### Asking LLM how to generate new tables in BRONZE layer and copy from RAW

Let's use the power of LLMs to translate column names into something meaningful for analyst and provide the SQL to crate new tables and copy the content from RAW ones. This will take a few minutes

In [ ]:
import re

table_names = ["FI_AR_4", "CUSTOMER", "MATERIAL"]

results = {}

for table in table_names:
    
    sql_text1 = f"""
    select snowflake.cortex.complete('{LLM}', '
    Generate a new SQL CREATE OR REPLACE table statement which will replace all 
    the below column names by easy and clear to understand column names for a 
    data analyst. Generate the SQL to copy the data from the source table to the
    target table respecting the column names. 
    The data types should be kept the same. Source schema is called {RAW_LAYER}. 
    Target schema is called {BRONZE_LAYER}. 
    It should in run Snowflake.'  
    || GET_DDL('table','{RAW_LAYER}."{table}"') );
    """

    full_text_str = session.sql(sql_text1).collect()[0][0]

    # Extract content between ```sql and ```
    match = re.search(r"```sql(.*?)```", full_text_str, re.DOTALL)
    if match:
        extracted_sql = match.group(1).strip()
        extracted_sql = extracted_sql.replace('"', '\"')  # Double double-quotes for SQL safety

    else:
        extracted_sql = '""" """'
        
    results[table] = {
        "table_name": table,
        "full_output": full_text_str,
        "extracted_sql": extracted_sql
    }

In [ ]:
# Visualize the results before executing. Click on each expander to review.
for table, data in results.items():
    with st.expander(f"Full Output for Table: {data['table_name']}"):
        st.subheader("Full Output:")
        st.code(data["full_output"], language="sql")

    with st.expander(f"SQL for Table: {data['table_name']}"):
        st.subheader("Extracted SQL:")
        st.code(data["extracted_sql"], language="sql")

If you are ok with the output, run next cell to extract the SQL provided to CREATE the new table and the one to INSERT the data:

In [ ]:
import re

create_sql_statements = ""

for table, data in results.items():
    extracted_sql = data["extracted_sql"]

    # Updated regex to ensure we capture full CREATE and INSERT statements ending in ');'
    sql_statements = re.split(r';\s*\n', extracted_sql.strip())
    
    # Extracting the CREATE TABLE and INSERT statements
    create_table_sql = sql_statements[0] + ";"
    insert_sql = sql_statements[1] + ";"
    
    print(f"CREATING TABLE: {table}")
    session.sql(create_table_sql).collect()
    create_sql_statements += create_table_sql
    
    print(f"INSERT INTO TABLE {table}:")
    session.sql(insert_sql).collect()

In [ ]:
-- One of the advantages of Snowflake Notebooks is that you can combine Python, SQL and Markdown! Let's see what tables we got in this layer:
show tables;

In [ ]:
table FI_AR_4 limit 5;

### Generating GOLD Layer

Let's use the power of LLMs to create a Data MART with the previous tables. This will contain the most important information joining the 3 tables:


In [ ]:
sql_text2 = f"""
select snowflake.cortex.complete('{LLM}', '
Generate a single SQL statement which creates a Data mart table called ACCOUNTS_RECEIVABLE_MART selecting 25 most representative columns and joining the key columns properly.
It should include a column CUSTOMER_NAME, a posting date and clearing date and other information for each financial document, as well as the information necessary to calculate Days Sales Outstanding per customer 
Create the new table under {GOLD_LAYER} schema.
These are the tables to be used:
{create_sql_statements}');
"""

full_text_str2 = session.sql(sql_text2).collect()[0][0]

# Extract content between ```sql and ```
match = re.search(r"```sql(.*?)```", full_text_str2, re.DOTALL)
if match:
    extracted_sql2 = match.group(1).strip()
    extracted_sql2 = extracted_sql2.replace('"', '\"')  # Double double-quotes for SQL safety

else:
    extracted_sql2 = '""" """'

In [ ]:
with st.expander(f"Full Explanation:"):
    st.code(full_text_str2, language="sql")

with st.expander(f"SQL to build Mart:"):
    st.code(extracted_sql2, language="sql")

In [ ]:
session.sql('create or replace schema SAP_GOLD').collect()
session.sql(extracted_sql2).collect()

In [ ]:
table accounts_receivable_mart;

### Semantic Model: Cortex Search and Cortex Analyst

Using Snowflake Snowsight UI, we are going to create the Semantic Model that will be used by Cortex Analys to allow business users ask questions in natural language.

As we have a large number of CUSTOMER_NAME distinct values, and Analyst may want to ask for any of them, we are going to use the integration of Cortex Search and Cortex Analyst. 

Cortex Search will be enabled on those columns, so Cortex Analyst can retrieval names when needed, without having to provide all possible names in the semantic file.

In [ ]:
session.sql(f''' 
CREATE OR REPLACE CORTEX SEARCH SERVICE {GOLD_LAYER}.CUSTOMER_NAME_SEARCH 
ON CUSTOMER_NAME 
WAREHOUSE = COMPUTE_WH  
TARGET_LAG = '1 day' 
EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0' 
AS (SELECT DISTINCT CUSTOMER_NAME FROM {GOLD_LAYER}.accounts_receivable_mart);
''').collect()

In [ ]:
extracted_sql3 = extracted_sql2.replace("'", '')  # handle exceptions by removing single quotes

sql_text3 = f"""
select snowflake.cortex.complete('{LLM}', ' Provide 5 examples of questions that can be asked to this data mart, with the associated SQL query to answer this question: {extracted_sql3}');
"""

full_text_str3 = session.sql(sql_text3).collect()[0][0]

In [ ]:
with st.expander("Questions:"):
    st.code(full_text_str3, language="sql")

Examples of questions :

what's the top 10 of customers who takes the longest to pay (clear a facture) on average ?

# Now Automate the transformations by using Dynamic Tables
Take the above create statements and copy them here to make a dynamic table with a lag. When creating a dynamic table, you define a query that specifies how data should be transformed from base objects. Snowflake then handles the refresh schedule of the dynamic table, automatically updating it with changes made to the base objects based on the query.

## Raw to Bronze

In [ ]:
USE SCHEMA SAP_BRONZE;
CREATE OR REPLACE DYNAMIC TABLE Dyn_FI_AR_4 (
    SPECIAL_GL_INDICATOR VARCHAR(16777216),
    DOCUMENT_TYPE VARCHAR(16777216),
    POSTING_KEY VARCHAR(16777216),
    FISCAL_VARIANT VARCHAR(16777216),
    DOCUMENT_DATE TIMESTAMP_NTZ(9),
    POSTING_DATE TIMESTAMP_NTZ(9),
    ENTRY_DATE TIMESTAMP_NTZ(9),
    CLEARING_DATE TIMESTAMP_NTZ(9),
    LAST_CHANGE_DATE TIMESTAMP_NTZ(9),
    NET_DUE_DATE TIMESTAMP_NTZ(9),
    DISCOUNT_DATE_1 TIMESTAMP_NTZ(9),
    PAYMENT_BLOCK VARCHAR(16777216),
    PAYMENT_TERMS_GROUP VARCHAR(16777216),
    DUNNING_BLOCK VARCHAR(16777216),
    DUNNING_KEY VARCHAR(16777216),
    DUNNING_LEVEL VARCHAR(16777216),
    LOCAL_CURRENCY VARCHAR(16777216),
    LOCAL_CURR_DEBIT FLOAT,
    LOCAL_CURR_CREDIT FLOAT,
    LOCAL_CURR_TAX FLOAT,
    CASH_DISCOUNT_AMT FLOAT,
    CURRENCY_CODE VARCHAR(16777216),
    DISCOUNT_DATE_2 TIMESTAMP_NTZ(9),
    BASELINE_DATE TIMESTAMP_NTZ(9),
    CASH_DISC_DAYS_1 FLOAT,
    CASH_DISC_DAYS_2 FLOAT,
    NET_PAYMENT_DAYS FLOAT,
    CASH_DISC_PERCENT_1 FLOAT,
    CASH_DISC_PERCENT_2 FLOAT,
    COUNTRY_CODE VARCHAR(16777216),
    PAYMENT_METHOD VARCHAR(16777216),
    PAYMENT_TERMS VARCHAR(16777216),
    DOC_CURR_DEBIT FLOAT,
    DOC_CURR_CREDIT FLOAT,
    DOC_CURR_TAX FLOAT,
    DISCOUNT_BASE_AMT FLOAT,
    DOC_CURR_CASH_DISC FLOAT,
    CHART_OF_ACCOUNTS VARCHAR(16777216),
    GL_ACCOUNT VARCHAR(16777216),
    GL_ACCOUNT_NUMBER VARCHAR(16777216),
    CUSTOMER_SUB_ID VARCHAR(16777216),
    CLEARING_DOC_NO VARCHAR(16777216),
    SECOND_LOCAL_CURR VARCHAR(16777216),
    THIRD_LOCAL_CURR VARCHAR(16777216),
    POSTING_PERIOD VARCHAR(16777216),
    PROJECT_CODE VARCHAR(16777216),
    DEBIT_CREDIT_IND VARCHAR(16777216),
    AMOUNT_IN_DOC_CURR FLOAT,
    CHANGE_MODE VARCHAR(16777216),
    ENTITY_COUNTER FLOAT,
    DML_STATUS VARCHAR(16777216),
    DELTA_TOKEN VARCHAR(16777216),
    GLUE_FETCH_SEQ VARCHAR(16777216),
    COMPANY_CODE VARCHAR(16777216),
    FISCAL_PERIOD VARCHAR(16777216),
    ACCOUNTING_DOC_NO VARCHAR(16777216),
    LINE_ITEM_NO VARCHAR(16777216),
    SUB_LEDGER_LINE_NO VARCHAR(16777216),
    STATUS_PARK_SAVE VARCHAR(16777216),
    CUSTOMER_ID VARCHAR(16777216),
    CREDIT_CONTROL_AREA VARCHAR(16777216),
    DUNNING_AREA VARCHAR(16777216),
    ACCOUNT_TYPE VARCHAR(16777216),
    REFERENCE_DOC_NO VARCHAR(16777216),
    REFERENCE_DOC_NO_REV VARCHAR(16777216),
    REFERENCE_FISCAL_YEAR VARCHAR(16777216),
    REFERENCE_LINE_ITEM VARCHAR(16777216),
    SALES_DOC_NO VARCHAR(16777216),
    REFERENCE_KEY_1 VARCHAR(16777216),
    REFERENCE_KEY_2 VARCHAR(16777216),
    REFERENCE_KEY_3 VARCHAR(16777216),
    LINE_ITEM_TEXT VARCHAR(16777216),
    NEGATIVE_POSTING VARCHAR(16777216),
    ARCHIVED_FLAG VARCHAR(16777216),
    SPECIAL_GL_TYPE VARCHAR(16777216),
    UPDATE_MODE VARCHAR(16777216),
    ASSIGNMENT_NUMBER VARCHAR(16777216),
    REFERENCE_TRANS_TYPE VARCHAR(16777216),
    REFERENCE_KEY VARCHAR(16777216),
    DOCUMENT_STATUS VARCHAR(16777216),
    LOCAL_CURR_AMOUNT FLOAT,
    SECOND_LOCAL_CURR_AMT FLOAT,
    THIRD_LOCAL_CURR_AMT FLOAT,
    FISCAL_YEAR VARCHAR(16777216)
)
TARGET_LAG = DOWNSTREAM
WAREHOUSE = COMPUTE_WH
REFRESH_MODE = AUTO

AS

SELECT 
    UMSKZ,
    BLART,
    BSCHL,
    FISCVAR,
    BLDAT,
    BUDAT,
    CPUDT,
    AUGDT,
    MADAT,
    NETDT,
    SK1DT,
    ZLSPR,
    RSTGR,
    MANSP,
    MSCHL,
    MANST,
    LCURR,
    DMSOL,
    DMHAB,
    DMSHB,
    SKNTO,
    WAERS,
    SK2DT,
    ZFBDT,
    ZBD1T,
    ZBD2T,
    ZBD3T,
    ZBD1P,
    ZBD2P,
    LAND1,
    ZLSCH,
    ZTERM,
    WRSOL,
    WRHAB,
    WRSHB,
    SKFBT,
    WSKTO,
    KTOPL,
    HKONT,
    SAKNR,
    FILKD,
    AUGBL,
    HWAE2,
    HWAE3,
    MONAT,
    PROJK,
    SHKZG,
    WRBTR,
    ODQ_CHANGEMODE,
    ODQ_ENTITYCNTR,
    DML_STATUS,
    DELTA_TOKEN,
    GLUE_FETCH_SQ,
    BUKRS,
    FISCPER,
    BELNR,
    BUZEI,
    UPOSZ,
    STATUSPS,
    KUNNR,
    KKBER,
    MABER,
    KOART,
    XBLNR,
    REBZG,
    REBZJ,
    REBZZ,
    VBELN,
    XREF1,
    XREF2,
    XREF3,
    SGTXT,
    XNEGP,
    XARCH,
    UMSKS,
    UPDMOD,
    ZUONR,
    AWTYP,
    AWKEY,
    BSTAT,
    DMBTR,
    DMBE2,
    DMBE3,
    GJAHR
FROM SAP_RAW.FI_AR_4;


In [ ]:
USE SCHEMA SAP_BRONZE;
CREATE OR REPLACE DYNAMIC TABLE Dyn_CUSTOMER (
    TAX_NUMBER_1 VARCHAR(16777216),
    ACCOUNT_NUMBER VARCHAR(16777216),
    STATE_TAX_NUMBER VARCHAR(16777216),
    REGION VARCHAR(16777216),
    STREET_ADDRESS VARCHAR(16777216),
    PHONE_NUMBER VARCHAR(16777216),
    CUSTOMER_ID VARCHAR(16777216),
    COUNTRY_CODE VARCHAR(16777216),
    COMPANY_NAME VARCHAR(16777216),
    COMPANY_NAME_2 VARCHAR(16777216),
    CITY VARCHAR(16777216),
    POSTAL_CODE VARCHAR(16777216),
    ACCOUNTING_START_DATE TIMESTAMP_NTZ(9),
    CHANGE_MODE VARCHAR(16777216),
    ENTITY_COUNTER FLOAT,
    DATA_STATUS VARCHAR(16777216),
    LOCAL_TAX_NUMBER VARCHAR(16777216),
    LAST_ACCOUNTING_DATE TIMESTAMP_NTZ(9),
    BUSINESS_TYPE VARCHAR(16777216),
    ANNUAL_REVENUE FLOAT,
    DELTA_TOKEN VARCHAR(16777216),
    FETCH_SEQUENCE VARCHAR(16777216)
)
TARGET_LAG = DOWNSTREAM
WAREHOUSE = COMPUTE_WH
REFRESH_MODE = AUTO

AS
SELECT 
    UWAER as TAX_NUMBER_1,
    JMZAH as ACCOUNT_NUMBER,
    J_1ICSTNO as STATE_TAX_NUMBER,
    REGIO as REGION,
    STRAS as STREET_ADDRESS,
    TELF1 as PHONE_NUMBER,
    KUNNR as CUSTOMER_ID,
    LAND1 as COUNTRY_CODE,
    NAME1 as COMPANY_NAME,
    NAME2 as COMPANY_NAME_2,
    ORT01 as CITY,
    PSTLZ as POSTAL_CODE,
    J_3GAABRECH as ACCOUNTING_START_DATE,
    ODQ_CHANGEMODE as CHANGE_MODE,
    ODQ_ENTITYCNTR as ENTITY_COUNTER,
    DML_STATUS as DATA_STATUS,
    J_1ILSTNO as LOCAL_TAX_NUMBER,
    J_3GLABRECH as LAST_ACCOUNTING_DATE,
    BRSCH as BUSINESS_TYPE,
    UMSAT as ANNUAL_REVENUE,
    DELTA_TOKEN,
    GLUE_FETCH_SQ as FETCH_SEQUENCE
FROM SAP_RAW.CUSTOMER;

In [ ]:

CREATE OR REPLACE DYNAMIC TABLE DYN_MATERIAL (
    IS_BATCH_MANAGED VARCHAR(16777216),
    IS_CROSS_PLANT_CONFIGURABLE VARCHAR(16777216),
    PRODUCT_CATEGORY VARCHAR(16777216),
    PRODUCT_CHARACTERISTIC_1 VARCHAR(16777216),
    MATERIAL_GROUP VARCHAR(16777216),
    BASE_UNIT VARCHAR(16777216),
    WEIGHT_UNIT VARCHAR(16777216),
    CHANGE_MODE VARCHAR(16777216),
    ENTITY_COUNTER FLOAT,
    STATUS VARCHAR(16777216),
    DELTA_TOKEN VARCHAR(16777216),
    FETCH_SEQUENCE VARCHAR(16777216),
    MATERIAL_CODE VARCHAR(16777216),
    MATERIAL_TYPE VARCHAR(16777216),
    MANUFACTURER_NUMBER VARCHAR(16777216),
    MANUFACTURER_PART_NUMBER VARCHAR(16777216),
    AUTH_GROUP VARCHAR(16777216),
    PRODUCT_CHARACTERISTIC_2 VARCHAR(16777216),
    PRODUCT_CHARACTERISTIC_3 VARCHAR(16777216)
) 
TARGET_LAG = DOWNSTREAM
WAREHOUSE = COMPUTE_WH
REFRESH_MODE = AUTO

AS 
SELECT 
    ISBATCHMANAGEMENTREQUIRED,
    CROSSPLANTCONFIGURABLEPRODUCT,
    PRODUCTCATEGORY,
    PRODCHARC1INTERNALNUMBER,
    MATERIALGROUP,
    MATERIALBASEUNIT,
    MATERIALWEIGHTUNIT,
    ODQ_CHANGEMODE,
    ODQ_ENTITYCNTR,
    DML_STATUS,
    DELTA_TOKEN,
    GLUE_FETCH_SQ,
    MATERIAL,
    MATERIALTYPE,
    MATERIALMANUFACTURERNUMBER,
    MATERIALMANUFACTURERPARTNUMBER,
    AUTHORIZATIONGROUP,
    PRODCHARC2INTERNALNUMBER,
    PRODCHARC3INTERNALNUMBER
FROM SAP_RAW.MATERIAL;

## Bronze to Gold

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE SAP_GOLD.DYN_ACCOUNTS_RECEIVABLE_MART 

TARGET_LAG ='60 seconds'
WAREHOUSE = COMPUTE_WH
REFRESH_MODE = AUTO

AS
SELECT 
    -- Document Information
    ar.ACCOUNTING_DOC_NO,
    ar.DOCUMENT_TYPE,
    ar.LINE_ITEM_NO,
    ar.REFERENCE_DOC_NO,
    ar.SALES_DOC_NO,

    -- Customer Information
    ar.CUSTOMER_ID,
    c.COMPANY_NAME as CUSTOMER_NAME,
    c.COUNTRY_CODE,
    c.REGION,
    --c.BUSINESS_SECTOR,

    -- Dates
    ar.DOCUMENT_DATE,
    ar.POSTING_DATE,
    ar.CLEARING_DATE,
    ar.NET_DUE_DATE,
    ar.BASELINE_DATE,

    -- Amount Fields
    --ar.AMOUNT_DOC_CURR,
    ar.LOCAL_CURR_AMOUNT,
    ar.LOCAL_CURR_DEBIT,
    ar.LOCAL_CURR_CREDIT,
    ar.CURRENCY_CODE,

    -- Payment Terms and Status
    ar.PAYMENT_TERMS,
    ar.NET_PAYMENT_DAYS,
    --ar.STATUS_PAYMENT,
    ar.PAYMENT_BLOCK,

    -- Additional Fields for Analysis
    ar.FISCAL_YEAR,
    ar.FISCAL_PERIOD,
    ar.COMPANY_CODE,
    ar.DEBIT_CREDIT_IND,
    ar.DOCUMENT_STATUS

FROM SAP_BRONZE.Dyn_FI_AR_4 ar
LEFT JOIN SAP_BRONZE.Dyn_CUSTOMER c 
    ON ar.CUSTOMER_ID = c.CUSTOMER_ID;

Remember to suspend your pipes and dynamic tables to reduce costs

In [ ]:
ALTER DYNAMIC TABLE IF EXISTS SAP_GOLD.DYN_ACCOUNTS_RECEIVABLE_MART SUSPEND;
ALTER DYNAMIC TABLE IF EXISTS SAP_BRONZE.DYN_MATERIAL SUSPEND;
ALTER DYNAMIC TABLE IF EXISTS SAP_BRONZE.CUSTOMER SUSPEND;
ALTER DYNAMIC TABLE IF EXISTS SAP_BRONZE.FI_AR_4 SUSPEND;

ALTER PIPE SAP_RAW.FI_AR_4_PIPE SET PIPE_EXECUTION_PAUSED = TRUE;
ALTER PIPE SAP_RAW.CUSTOMER_PIPE SET PIPE_EXECUTION_PAUSED = TRUE;
ALTER PIPE SAP_RAW.MATERIAL_PIPE SET PIPE_EXECUTION_PAUSED = TRUE;

